In [2]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import random 
from nltk.corpus import stopwords
import string
import nltk
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('C:\\Users\\vatss\\codes\\TwitterSentimentAnalysis\\train.csv')
test = pd.read_csv('C:\\Users\\vatss\\codes\\TwitterSentimentAnalysis\\test.csv')
train
# import resource
# soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
# resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [21]:
text_train_array = train.text.str.split()
text_test_array = test.text.str.split()
target_train = train.airline_sentiment

In [22]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vatss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def get_simple_pos(pos):
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('R'):
        return wordnet.ADV
    else: return wordnet.NOUN

In [11]:
lemmatizer = WordNetLemmatizer()
def clean_data(words):
    clean_words=[]
    for w in set(words):
        if w.lower() not in stops:
            pos = pos_tag([w])[0][1]
            clean_word = lemmatizer.lemmatize(w,pos=get_simple_pos(pos))
            clean_words.append(clean_word.lower())
    return clean_words

In [14]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
train_text_cleaned = [clean_data(words) for words in text_train_array]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vatss\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vatss\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [15]:
test_text_cleaned = [clean_data(words) for words in text_test_array]

0        negative
1        positive
2        positive
3        negative
4        negative
           ...   
10975     neutral
10976    positive
10977    negative
10978    negative
10979    negative
Name: airline_sentiment, Length: 10980, dtype: object

In [17]:
X_train=[]
X_test=[]
for List in train_text_cleaned:
    X_train.append(" ".join(List))
for List in test_text_cleaned:
    X_test.append(" ".join(List))

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.linear_model import LogisticRegression

In [66]:
count_vec = CountVectorizer(max_features=925)
X = count_vec.fit_transform(X_train)
Test = count_vec.transform(X_test)
X = X.todense()
Test = Test.todense()
enc = LabelEncoder()
Y=enc.fit_transform(target_train)

In [67]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
Test = scaler.transform(Test)

In [68]:
clf = LogisticRegression(solver='lbfgs')
clf.fit(X,Y)
result = clf.predict(Test)
final = enc.inverse_transform(result)
pd.DataFrame(final).to_csv("pred.csv",header=False,index=False)

/home/shivam/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
